In [1]:
pip install ccdproc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.4/355.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.5/368.5 kB 29.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 99.0 MB/s eta 0:00:00
  Created wheel for astroscrappy: filename=astroscrappy-1.1.0-cp310-cp310-linux_x86_64.whl size=1204038 sha256=297406050d9a8177a25ce555c7d4fb4137692e078f0bfe33066a0c81fdf82f23
  Stored in directory: /root/.cache/pip/wheels/59/b5/13/37c096a227ab03539bf8786d15a5d5789a686f86d284b5eb19
  Created wheel for asciitree: filename=asciitree-0.

In [2]:
pip install specutils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 957.5/957.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.2 MB/s eta 0:00:00


In [3]:
import sys
import scipy
import glob
import ccdproc
import matplotlib.pyplot as plt
from astropy import units as u
import numpy as np
import pandas as pd
from astropy.nddata import CCDData
from astropy.nddata import NDData
from astropy.nddata import fits_ccddata_reader
from astropy.nddata import fits_ccddata_writer
from astropy.modeling.functional_models import Gaussian2D
from astropy.utils.misc import NumpyRNGContext
from scipy.ndimage import uniform_filter
import astropy.io.fits as fits
from ccdproc import Combiner
from ccdproc import wcs_project
from astropy.io.fits import Header
from astropy import  wcs
from astropy.io import fits
from astropy.table import Table, Column
from specutils import Spectrum1D
import os
from specutils.manipulation import extract_region
from specutils import SpectralRegion

from astropy.modeling import models
from numpy.polynomial import polynomial as P
from astropy.modeling import models, fitting
from scipy import interpolate
from astropy.modeling.fitting import LevMarLSQFitter

from specutils.manipulation import extract_region
from specutils.fitting import fit_generic_continuum

from specutils.analysis import equivalent_width, centroid
from specutils import SpectralRegion

In [7]:
dir="/content/drive/MyDrive/Colab Notebooks/IC348N/Fits-91523202/"
Tabla = pd.DataFrame(columns = ['RA', 'DEC', 'EW_HA'])

for fitsName in glob.glob(dir+'*.fits.gz'):
    print(fitsName)


    hdul = fits.open(fitsName)
  #  hdul.info()

    specdata = hdul[0]

#    print(10**specdata.data[1],specdata.data[0])

    scidata = specdata.data[0]* u.dimensionless_unscaled

    lamb = specdata.data[2] * u.AA


    with open(fitsName+".txt",'w') as f:
        for row in zip(lamb, scidata):
            f.write('{}\t{}\n'.format(row[0], row[1]))


# To plot using maptplotlib:

    spec = Spectrum1D(spectral_axis=lamb, flux=scidata)

    cut_spectrum1_HA = extract_region(spec, SpectralRegion(6540.0*u.AA, 6550.0*u.AA))
    cut_spectrum2_HA = extract_region(spec, SpectralRegion(6575.0*u.AA, 6600.0*u.AA))
    cut_spectrumHA = extract_region(spec, SpectralRegion(6540.0*u.AA, 6590.0*u.AA))


    model_poly = models.Polynomial1D(degree=2)
    fitter_poly = fitting.LinearLSQFitter()
    best_fit_HA = fitter_poly(model_poly, np.concatenate((cut_spectrum1_HA.spectral_axis.value, cut_spectrum2_HA.spectral_axis.value), axis = None), np.concatenate((cut_spectrum1_HA.flux.value, cut_spectrum2_HA.flux.value), axis = None))
    nspec1d_HA = Spectrum1D(flux=cut_spectrumHA.flux.value/best_fit_HA(cut_spectrumHA.spectral_axis.value)*u.dimensionless_unscaled, spectral_axis=cut_spectrumHA.spectral_axis)


    anchitoHA=5
# First we compute the centroids:
    cHA = centroid(nspec1d_HA, SpectralRegion(6555*u.AA, 6575*u.AA))
# Thus we configure the width measured from the centroid:
    izq1HA=cHA - anchitoHA*u.AA
    der1HA=cHA + anchitoHA*u.AA
# we use photutils to fit a gaussian and get the equivalent width
    EW_HA = equivalent_width(nspec1d_HA,regions=SpectralRegion(izq1HA,der1HA))

    print(hdul[0].header['RA'],hdul[0].header['DEC'],EW_HA)
    ra = hdul[0].header['RA']
    dec = hdul[0].header['DEC']
    ew_ha = EW_HA
    Tabla = Tabla.append({'RA':ra, 'DEC':dec, 'EW_HA':ew_ha}, ignore_index=True)

    plt.plot(nspec1d_HA.spectral_axis, nspec1d_HA.flux)
    #plt.title('SSH (m) ' + str(ecco_ds.time[0].values)[0:7] + '\n tile 2')
    plt.title(fitsName,fontsize=10)

    plt.xlabel('Wavelength (\AA)')
    plt.ylabel('Relative Flux')
    plt.xlim(6540,6580) #Ha
   # plt.xlim(658,750) #OI

#plt.ylim(1e-7,1e1);
#    plt.savefig(str(fitsName)+".png")

    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(Tabla)
Tabla.to_csv('/content/drive/MyDrive/Colab Notebooks/IC348S/TablaEWHA.csv', index= False)

            RA        DEC                        EW_HA
0    56.368372  31.550316  2.2068553927814505 Angstrom
1    56.419149  31.626683   1.543211465411357 Angstrom
2    56.226372  31.489815    4.00212021984092 Angstrom
3    56.163324  31.747433  2.2841129846765664 Angstrom
4    56.189258  31.850977   4.934712508399478 Angstrom
..         ...        ...                          ...
97   56.250122  31.849320   0.896962763647082 Angstrom
98   56.247166  31.549811  1.4233928216487688 Angstrom
99   56.031342  31.599837  1.5608931923781149 Angstrom
100  56.348059  31.546432  2.2165339553774834 Angstrom
101  56.234381  31.514866  2.6706185949308336 Angstrom

[102 rows x 3 columns]
